In [1]:
import sys

sys.path.append("..")

import tensorflow as tf

import numpy as np

from utils import tf_config

dataset="01"
memory_limit=1500
n_individuals=10
interval_eeg=10

tf_config.set_seed(seed=3)
tf_config.setup_tensorflow(device="GPU", memory_limit=memory_limit)

import GPyOpt

import argparse

from utils import preprocess_data, search_algorithms, train, losses_utils, metrics, eeg_utils, fmri_utils

from models import fmri_ae, eeg_to_fmri, uniconv_fmri

from layers import locally_connected

import matplotlib.pyplot as plt

import gc

import os

from sklearn.model_selection import train_test_split, KFold

from pathlib import Path

import time

import pickle

2022-05-23 15:30:24.644892: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-05-23 15:30:25.912380: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-23 15:30:25.912858: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-23 15:30:25.960008: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-23 15:30:25.960571: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:0a:00.0 name: GeForce RTX 2080 SUPER computeCapability: 7.5
coreClock: 1.845GHz coreCount: 48 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 462.00GiB/s
2022-05-23 15:30:25.960605: I tensorflow/stream_executor/platform/def

## Build model

In [3]:
from benchmarks import liu

import importlib
importlib.reload(liu)
interval_eeg=2

with tf.device('/CPU:0'):
    model = liu.Liu_et_al((len(getattr(eeg_utils, "channels_"+dataset)),interval_eeg),
                          getattr(fmri_utils, "fmri_shape_"+dataset)+(interval_eeg,),
                         variational=True)
    
    optimizer = tf.keras.optimizers.Adam(0.001)
    loss_fn = model.get_loss()

    train_set, test_set = model.load_data(dataset, time_length=interval_eeg, batch_size=2)

## Train

In [4]:
loss_history = train.train(train_set, model, optimizer, loss_fn, epochs=10, u_architecture=False, verbose=True)[0]

2022-05-23 15:31:21.649365: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-05-23 15:31:21.931804: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-05-23 15:31:32.104585: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 480.00MiB (rounded to 503316480)requested by op Softplus
Current allocation summary follows.
2022-05-23 15:31:32.104625: I tensorflow/core/common_runtime/bfc_allocator.cc:972] BFCAllocator dump for GPU_0_bfc
2022-05-23 15:31:32.104638: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (256): 	Total Chunks: 55, Chunks in use: 55. 13.8KiB allocated for chunks. 13.8KiB in use in bin. 3.1KiB client-requested in use in bin.
2022-05-23 15:31:32.104646: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B all

ResourceExhaustedError: OOM when allocating tensor with shape[1,1024,122880] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Softplus]

## Metrics

In [ ]:
importlib.reload(metrics)
rmse_pop = metrics.rmse(test_set, model)
ssim_pop = metrics.ssim(test_set, model)
print("RMSE: ", np.mean(rmse_pop), "\pm", np.std(rmse_pop))
print("SSIM: ", np.mean(ssim_pop), "\pm", np.std(ssim_pop))

In [ ]:
1-
(01)
RMSE:  0.45489287 \pm 0.080613315
SSIM:  0.4488131258425048 \pm 0.06009989398335106


(02)
RMSE:  5458.107 \pm 2465.1108
SSIM:  0.12565268065278185 \pm 0.11898148319732271

(03)

